# Topic Modelling


Collect randomly as many as news content from different fields for training. (thru any form, web scrape, pdf, newspaper etc.)


Objective:-Build topic modelling to understand the topics 

<b>Imported the package Newspaper for scrapping the newspaper articles</b>

In [108]:
import newspaper
 
#Using Times of India for scrapping the news    
site = newspaper.build("https://timesofindia.indiatimes.com/news", memoize_articles=False)  
 
# get list of article URLs
#site.article_urls()

In [103]:
site_article = site.articles[0:50]

In [104]:
site_article

In [105]:
#getting the articles
article_list=[]
for i in site_article:
    i.download()
    i.parse()
    article_list.append(i.text)

<b>Printing the sample news </b>

In [106]:
article_list[5]

'The indiatimes.com privacy policy has been updated to align with the new data regulations in European Union. Please review and accept these changes below to continue using the website. We use cookies to ensure the best experience for you on our website.'

# Preprocessing

In [52]:
#Data Preprocessing
#Tokenization
import warnings
warnings.filterwarnings("ignore")
'''
Loading gensim and nltk libraries
'''
#!pip install gensim
import gensim

# Converts into tokens (Alternative to word_tokenize)
from gensim.utils import simple_preprocess

from gensim.parsing.preprocessing import STOPWORDS

from nltk.stem import WordNetLemmatizer, SnowballStemmer
#from nltk.stem import *
import numpy as np
np.random.seed(400)

# Lemmatizer Test

In [53]:
WordNetLemmatizer().lemmatize('runs')

'run'

# Stemming Test

In [54]:
#testing the stemming part before preprocessing. This should be changing any plural into singular word
import pandas as pd
stemmer = SnowballStemmer("english")
original_words=['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']

singles= [WordNetLemmatizer().lemmatize(plural,pos='v') for plural in original_words]

pd.DataFrame(data={'Original Words':original_words, 'Lemma':singles})

#Stemma is not performing well 
#singles= [stemmer.stem(plural) for plural in original_words]
#singles2= [stemmer.stem(plural2) for plural2 in singles]
#stemma = pd.DataFrame(data={'Lemma':singles, 'Stemmed':singles2})

,Original Words,Lemma
0,caresses,caress
1,flies,fly
2,dies,die
3,mules,mules
4,denied,deny
5,died,die
6,agreed,agree
7,owned,own
8,humbled,humble
9,sized,size


In [55]:
# writing function for the entire dataset
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))

#Tokenize and Lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token)>3:
            result.append(lemmatize_stemming(token))
            
    return result
        

In [56]:
#preview a document before preprocessing

doc_sample = 'This disk has failed many times. I would like to get it replaced.'

print("Original Document: ")
words=[]
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original Document: 
['This', 'disk', 'has', 'failed', 'many', 'times.', 'I', 'would', 'like', 'to', 'get', 'it', 'replaced.']


Tokenized and lemmatized document: 
['disk', 'fail', 'time', 'like', 'replac']


In [57]:
processed_docs = []

for doc in article_list:
    processed_docs.append(preprocess(doc))

In [58]:
len(processed_docs)

50

In [59]:
print(processed_docs[:2])

[['dhoni', 'genius', 'say', 'curran', 'england', 'rounder', 'curran', 'bat', 'ahead', 'mahendra', 'singh', 'dhoni', 'chennai', 'super', 'king', 'mumbai', 'indian', 'open', 'fixtur', 'indian', 'premier', 'leagu', 'saturday', 'england', 'rounder', 'leav', 'surpris', 'genius', 'skipper', 'qaida', 'cell', 'plot', 'hit', 'hold', 'nation', 'investig', 'agenc', 'saturday', 'bust', 'pakistan', 'sponsor', 'qaida', 'modul', 'oper', 'west', 'bengal', 'kerala', 'arrest', 'person', 'plan', 'carri', 'terrorist', 'strike', 'india', 'near', 'futur', 'sophist', 'weapon', 'arrang', 'qaida', 'handler', 'hamza', 'base', 'peshawar', 'pakistan', 'baseless', 'kashyap', 'sexual', 'abus', 'alleg', 'anurag', 'kashyap', 'take', 'twitter', 'respond', 'payal', 'ghosh', 'sexual', 'harass', 'accus', 'call', 'baseless', 'write', 'water', 'power', 'bill', 'act', 'input', 'receiv', 'public', 'outreach', 'take', 'charg', 'month', 'lieuten', 'governor', 'manoj', 'sinha', 'saturday', 'unveil', 'crore', 'econom', 'packag',

# Bag Of Words Featurisation

In [60]:
#bag of words on the data set
dictionary = gensim.corpora.Dictionary(processed_docs)

In [61]:
print (dictionary)

Dictionary(1727 unique tokens: ['absolut', 'abus', 'accord', 'accus', 'act']...)


In [62]:
#Testing! if the  dictionary created succesfully
count=0
for k,v in dictionary.iteritems():
    print (k, v)
    count +=1
    if count >20:
        break

0 absolut
1 abus
2 accord
3 accus
4 act
5 action
6 activ
7 actor
8 administ
9 agenc
10 ahead
11 alert
12 alleg
13 amend
14 anurag
15 area
16 armi
17 arrang
18 arrest
19 arunach
20 asapila


In [64]:
print (dictionary)

Dictionary(1727 unique tokens: ['absolut', 'abus', 'accord', 'accus', 'act']...)


In [65]:
#Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
#words and how many times those words appear. Save this to 'bow_corpus'
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [109]:
#bow_corpus[0]

In [67]:
#preview 
document_num = 10
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]],
                                                     bow_doc_x[i][1]))

Word 9 ("agenc") appears 1 time.
Word 54 ("come") appears 2 time.
Word 59 ("conduct") appears 1 time.
Word 114 ("futur") appears 1 time.
Word 134 ("hold") appears 1 time.
Word 141 ("indian") appears 2 time.
Word 147 ("intern") appears 1 time.
Word 168 ("level") appears 1 time.
Word 215 ("plan") appears 1 time.
Word 218 ("point") appears 1 time.
Word 256 ("say") appears 1 time.
Word 257 ("school") appears 1 time.
Word 266 ("serv") appears 1 time.
Word 298 ("tell") appears 1 time.
Word 335 ("work") appears 1 time.
Word 339 ("year") appears 1 time.
Word 392 ("father") appears 2 time.
Word 444 ("place") appears 1 time.
Word 458 ("south") appears 2 time.
Word 462 ("telangana") appears 1 time.
Word 477 ("news") appears 1 time.
Word 479 ("thing") appears 1 time.
Word 485 ("continu") appears 1 time.
Word 497 ("add") appears 1 time.
Word 530 ("imag") appears 1 time.
Word 610 ("encourag") appears 1 time.
Word 625 ("food") appears 1 time.
Word 709 ("student") appears 1 time.
Word 729 ("want") app

# Applying LDA modelling

In [68]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 10, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

# Topic Modelling

In [69]:
for idx, topic in lda_model.print_topics():
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.034*"oneplus" + 0.015*"price" + 0.014*"list" + 0.013*"best" + 0.013*"month" + 0.013*"seri" + 0.012*"year" + 0.011*"updat" + 0.009*"nord" + 0.008*"movi"


Topic: 1 
Words: 0.021*"releas" + 0.021*"film" + 0.018*"best" + 0.011*"list" + 0.010*"ethylen" + 0.010*"dioxid" + 0.010*"carbon" + 0.010*"reaction" + 0.010*"year" + 0.009*"date"


Topic: 2 
Words: 0.020*"say" + 0.014*"polic" + 0.011*"paint" + 0.011*"peopl" + 0.010*"chines" + 0.010*"simmon" + 0.007*"india" + 0.007*"facebook" + 0.007*"ashna" + 0.006*"school"


Topic: 3 
Words: 0.001*"oneplus" + 0.001*"year" + 0.001*"say" + 0.001*"price" + 0.001*"cinema" + 0.001*"help" + 0.001*"saturday" + 0.001*"malayalam" + 0.001*"india" + 0.001*"month"


Topic: 4 
Words: 0.020*"latest" + 0.014*"movi" + 0.014*"road" + 0.010*"cinema" + 0.009*"happen" + 0.009*"know" + 0.009*"announc" + 0.008*"daili" + 0.008*"want" + 0.008*"accid"


Topic: 5 
Words: 0.012*"india" + 0.009*"say" + 0.009*"agricultur" + 0.008*"saturday" + 0.007*"indian" + 0

# Performing Topic Modeling on the test data set

In [90]:
article_list = site.articles[51:60]
article_list_test=[]
for i in article_list:
    i.download()
    i.parse()
    article_list_test.append(i.text)

In [94]:
article_list_test[8]

'The Rail Land Development Authority (RLDA) has invited bids from private players to redevelop an area of five lakh square metres at the New Delhi station and another 2.6 lakh square metres surrounding it for commercial purposes. The organisation said that the project was likely to be completed in around four years. The plan is to redevelop the iconic station, road connections through flyovers, relocation of railway offices and buildings and creating social infrastructure. The estimated cost of the entire project is Rs 6,500 crore.'

In [95]:
bow_vector = dictionary.doc2bow(preprocess(article_list_test[8]))

for index, score in sorted(lda_model[bow_vector]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 30)))

Score: 0.08916326612234116	 Topic: 0.034*"oneplus" + 0.015*"price" + 0.014*"list" + 0.013*"best" + 0.013*"month" + 0.013*"seri" + 0.012*"year" + 0.011*"updat" + 0.009*"nord" + 0.008*"movi" + 0.008*"bengali" + 0.008*"discount" + 0.008*"cost" + 0.008*"sale" + 0.008*"option" + 0.008*"flagship" + 0.008*"offer" + 0.008*"content" + 0.008*"right" + 0.007*"time" + 0.006*"decis" + 0.006*"train" + 0.006*"smartphon" + 0.006*"storag" + 0.006*"season" + 0.006*"drop" + 0.006*"deal" + 0.006*"attent" + 0.005*"cast" + 0.005*"rat"
Score: 0.11471286416053772	 Topic: 0.020*"say" + 0.014*"polic" + 0.011*"paint" + 0.011*"peopl" + 0.010*"chines" + 0.010*"simmon" + 0.007*"india" + 0.007*"facebook" + 0.007*"ashna" + 0.006*"school" + 0.006*"come" + 0.006*"larg" + 0.006*"sharma" + 0.006*"intellig" + 0.006*"arrest" + 0.006*"parti" + 0.004*"indian" + 0.004*"educ" + 0.004*"seri" + 0.004*"want" + 0.004*"hold" + 0.004*"nation" + 0.004*"kumar" + 0.004*"document" + 0.004*"inform" + 0.004*"delhi" + 0.004*"citi" + 0.004*